In [6]:
from pytket.extensions.qiskit.backends.aer import AerBackend
from pytket.circuit import BasisOrder, Circuit

backend = AerBackend()

c = Circuit(2, 2)
c.H(0)
c.CX(0, 1)
c.measure_all()

c = backend.get_compiled_circuit(c)
handle = backend.process_circuit(c, n_shots=10)
counts = backend.get_result(handle).get_counts()
print(counts)

Counter({(0, 0): 8, (1, 1): 2})


In [7]:
backend.get_result(handle).get_counts()

Counter({(0, 0): 8, (1, 1): 2})

In [8]:
backend.get_result(handle).get_shots()

array([[0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [1, 1],
       [1, 1],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0]], dtype=uint8)

In [9]:
test_handle = backend.get_result(handle)

In [10]:
test_handle.c_bits

{c[0]: 0, c[1]: 1}

# Simplest example

In [39]:
from pytket.extensions.qiskit.backends.aer import AerBackend
from pytket.circuit import BasisOrder, Circuit

backend = AerBackend()

c = Circuit(1,1)
c.H(0)
c.measure_all()

c = backend.get_compiled_circuit(c)
handle = backend.process_circuit(c, n_shots=10)
counts = backend.get_result(handle).get_counts()
print(counts)

Counter({(1,): 6, (0,): 4})


In [40]:
test_handle = backend.get_result(handle)

In [41]:
test_handle.get_shots()

array([[1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1]], dtype=uint8)

In [17]:
from pytket.utils.outcomearray import OutcomeArray

In [ ]:
OutcomeArray.from_readouts(bit_strings

In [16]:
test_handle._shots

OutcomeArray([[128],
              [  0],
              [  0],
              [128],
              [  0],
              [  0],
              [128],
              [128],
              [128],
              [128]], dtype=uint8)

In [44]:
seed = 3
import numpy as np
n_shots = 10000

rng = np.random.default_rng(seed)
randnums = rng.random(n_shots).tolist()

In [4]:
# Plot the random numbers as bar chart coarse grained
import matplotlib.pyplot as plt
plt.hist(randnums, bins=100, edgecolor='black')
plt.xlabel('Random Number')
plt.ylabel('Frequency')
plt.title('Random Numbers Histogram')
plt.show()

: 

In [10]:
type(randnums[0])

float

In [11]:
print(f"randnums dtype: {type(randnums[0])}")
print(f"randnums min: {np.min(randnums)}")
print(f"randnums max: {np.max(randnums)}")
# Check if any values are exactly 1.0 (shouldn't be)
print(f"randnums contains 1.0: {1.0 in randnums}")

randnums dtype: <class 'float'>
randnums min: 0.001148413743897403
randnums max: 0.9998030282851982
randnums contains 1.0: False


In [ ]:
import numpy as np
import cupy as cp
import cuquantum.custatevec as cusv
from cuquantum import cudaDataType

n_qubits = 1
n_shots = 10000

handle = cusv.create()
# |0>
sv = cp.zeros(2, dtype=cp.complex128)
sv[0] = 1.0

# Apply Hadamard manually
H = 1/np.sqrt(2)*cp.array([[1,1],[1,-1]], dtype=cp.complex128)
sv[:] = H @ sv

# Create sampler
sampler, workspace_size = cusv.sampler_create(handle, sv.data.ptr, cudaDataType.CUDA_C_64F, n_qubits, n_shots)
workspace = cp.cuda.alloc(workspace_size)
cusv.sampler_preprocess(
        handle,
        sampler,
        workspace.ptr,
        workspace_size,
    )

bit_strings = np.empty(n_shots, dtype=np.int32)
randnums = np.linspace(0, 1, n_shots, endpoint=False, dtype=np.float64)


cusv.sampler_sample(
    handle,
    sampler,
    bit_strings.ctypes.data,
    [0], # bit_ordering
    1,   # bit_string_len
    randnums,
    n_shots,
    cusv.SamplerOutput.RANDNUM_ORDER,
)

cusv.sampler_destroy(sampler)

print(np.mean(bit_strings))  # Should be ~0.5

: 

In [2]:
import numpy as np
import cupy as cp
import cuquantum.custatevec as cusv
from cuquantum import cudaDataType

n_qubits = 1
n_shots = 10000

handle = cusv.create()
try:
    sv = cp.zeros(2, dtype=cp.complex128)
    sv[0] = 1.0

    H = (1/np.sqrt(2)) * cp.array([[1, 1], [1, -1]], dtype=cp.complex128)
    sv[:] = H @ sv

    print("Statevector:", cp.asnumpy(sv))

    sampler, workspace_size = cusv.sampler_create(
        handle,
        sv.data.ptr,
        cudaDataType.CUDA_C_64F,
        n_qubits,
        n_shots
    )
    workspace = cp.cuda.alloc(workspace_size)

    cusv.sampler_preprocess(
        handle,
        sampler,
        workspace.ptr,
        workspace_size
    )

    bit_ordering = [0]
    print("bit_ordering:", bit_ordering)

    randnums = np.random.default_rng().random(n_shots, dtype=np.float64)
    randnums = np.sort(randnums)  # Sort to ensure uniform distribution
    print("randnums dtype:", randnums.dtype, "min:", randnums.min(), "max:", randnums.max())

    bit_strings = np.empty(n_shots, dtype=np.int32)
    cusv.sampler_sample(
        handle,
        sampler,
        bit_strings.ctypes.data,
        bit_ordering,
        len(bit_ordering),
        randnums,
        n_shots,
        cusv.SamplerOutput.ASCENDING_ORDER,
    )

    cusv.sampler_destroy(sampler)

    print("cuStateVec counts:", np.bincount(bit_strings))

    # Manual sampling sanity check
    probs = cp.abs(sv)**2
    manual_samples = np.random.choice([0,1], size=n_shots, p=cp.asnumpy(probs))
    print("Manual counts:", np.bincount(manual_samples))

finally:
    cusv.destroy(handle)


: 

In [ ]:
import matplotlib.pyplot as plt
plt.hist(randnums, bins=100, edgecolor='black')
plt.xlabel('Random Number')
plt.ylabel('Frequency')
plt.title('Random Numbers Histogram')
plt.show()

array([0, 0, 0, ..., 0, 0, 0], shape=(10000,), dtype=int32)

In [4]:
from collections import Counter
Counter(bit_strings)

Counter({np.int32(0): 9971, np.int32(1): 29})

In [3]:
randnums

array([4.63317968e-05, 5.85458166e-05, 7.67083663e-05, ...,
       9.99648033e-01, 9.99835320e-01, 9.99988637e-01], shape=(10000,))